# v12 (continue training)    

name : melanoma_tachyon_v12  
about : check qishen ha model.  
model : Enet (b3) batchnorm freeze  
img_size : 384x384  
batch : 16  
epoch : 4  
criterion : BCEWithLogitsLoss  
optimizer : RAdam  
init_lr : 2e-5  
scheduler: CosineAnnealingLR  
data : melanoma-merged-external-data-512x512-jpeg-shades  
preprocess : Shades  
train_test_split : 5 of 3 folds(Triple StratifiedKFold, k=5)  
data augmentation : hairaug,flip,crop,jitter,cutout  
tta : 3 times(same condition of train DA)  

In [ ]:
local = False
DEBUG = False

In [ ]:
#!pip install efficientnet_pytorch torchtoolbox

In [ ]:
#!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

In [ ]:
#before import process
import sys
if local == True:
    package_path = 'Q:/kaggle/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
else:
    package_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(package_path)

#imports
import os, warnings, random, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from IPython.display import display
from tqdm import tqdm_notebook as tqdm
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.functional import F 
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import model as enet
import albumentations as A
#import torchtoolbox.transform as transforms
#from warmup_scheduler import GradualWarmupScheduler
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
SEED = 32 #69
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


In [ ]:
#params
enet_type = 'efficientnet-b3'
model_name = 'v12'
n_epochs = 5 if DEBUG else 4
cosine_t = 6
n_fold = 5

batch_size = 32
image_size = 300

num_workers = 2

init_lr = 2e-5
TTA = 3

Progress_Bar = False

In [ ]:
if local:
    pass
else:
    train_df = pd.read_csv('../input/melanoma-merged-external-data-512x512-jpeg/folds_13062020.csv')
    test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
    train_images = '../input/melanoma-merged-external-data-512x512-jpeg/512x512-dataset-melanoma/512x512-dataset-melanoma'
    test_images = '../input/melanoma-merged-external-data-512x512-jpeg/512x512-test/512x512-test'
    model_dir = '../input/melanoma-tachyon-v12-models'

if DEBUG:
    train_df = train_df[:100]
    test_df = test_df[:50]
    
# One-hot encoding of anatom_site_general_challenge feature
concat = pd.concat([train_df['anatom_site_general_challenge'], test_df['anatom_site_general_challenge']], ignore_index=True)
dummies = pd.get_dummies(concat, dummy_na=True, dtype=np.uint8, prefix='site')
train_df = pd.concat([train_df, dummies.iloc[:train_df.shape[0]]], axis=1)
test_df = pd.concat([test_df, dummies.iloc[train_df.shape[0]:].reset_index(drop=True)], axis=1)

# Sex features
train_df['sex'] = train_df['sex'].map({'male': 1, 'female': 0})
test_df['sex'] = test_df['sex'].map({'male': 1, 'female': 0})
train_df['sex'] = train_df['sex'].fillna(-1)
test_df['sex'] = test_df['sex'].fillna(-1)

# Age features
train_df['age_approx'] /= train_df['age_approx'].max()
test_df['age_approx'] /= test_df['age_approx'].max()
train_df['age_approx'] = train_df['age_approx'].fillna(0)
test_df['age_approx'] = test_df['age_approx'].fillna(0)

train_df['patient_id'] = train_df['patient_id'].fillna(0)

In [ ]:
train_df.head()

In [ ]:
"""train_df.reset_index(inplace=True)
skf = StratifiedKFold(n_fold, shuffle=True, random_state=SEED)
train_df['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df['target'])):
    train_df.loc[valid_idx, 'fold'] = i"""

In [ ]:
meta_features = ['sex', 'age_approx'] + [col for col in train_df.columns if 'site_' in col]
meta_features.remove('anatom_site_general_challenge')

In [ ]:
if local:
    pretrained_model = {
        'efficientnet-b0': 'Q:/kaggle/efficientnet-pytorch/efficientnet-b0-08094119.pth'
        
    }
    model_save_path = 'Q:/kaggle/melanoma-trained-models'
else:
    pretrained_model = {
        'efficientnet-b0': '../input/efficientnet-pytorch/efficientnet-b0-08094119.pth',
        'efficientnet-b1': '../input/efficientnet-pytorch/efficientnet-b1-dbc7070a.pth',
        'efficientnet-b2': '../input/efficientnet-pytorch/efficientnet-b2-27687264.pth',
        'efficientnet-b3': '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth',
        'efficientnet-b4': '../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth',
        'efficientnet-b5': '../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth',
        
    }
    model_save_path = False

class enetv2(nn.Module):
    def __init__(self, backbone, out_dim, n_meta_features):
        super(enetv2, self).__init__()
        if out_dim != 500:
            out_dim = 500
            print("out_dim size is changed")
        
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))

        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.meta = nn.Sequential(nn.Linear(n_meta_features, 500),
                                  nn.BatchNorm1d(500),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2),
                                  nn.Linear(500, 250),  # FC layer output will have 250 features
                                  nn.BatchNorm1d(250),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2))
        self.sigmoid = nn.Sigmoid()
        self.ouput = nn.Linear(500 + 250, 1)
        
    def extract(self, x):
        return self.enet(x)

    def forward(self, inputs):
        x,meta = inputs
        
        x = self.extract(x)
        x = self.myfc(x) #500
        
        meta = self.meta(meta) #250
        
        features = torch.cat((x, meta), dim=1)
        output = self.ouput(features)
        #output = self.sigmoid(output) #no sigmoid(if BCE loss)
        return output 
def load_models(model_files):
    models = []
    for model_f in model_files:
        model_f = os.path.join(model_dir, model_f)
        model = enetv2(enet_type, out_dim=500,n_meta_features = len(meta_features))
        model.load_state_dict(torch.load(model_f, map_location=lambda storage, loc: storage), strict=True)
        model.eval()
        model.to(device)
        models.append(model)
        print(f'{model_f} loaded!')
    return models


model_files = [
    'v12_efficientnet-b3_f0_final.pt'
]

pre_models = load_models(model_files)

In [ ]:
class AdvancedHairAugmentation:
    """
    Impose an image of a hair to the target image

    Args:
        hairs (int): maximum number of hairs to impose
        hairs_folder (str): path to the folder with hairs images
    """

    def __init__(self, hairs: int = 5, hairs_folder: str = ""):
        self.hairs = hairs
        self.hairs_folder = hairs_folder

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to draw hairs on.

        Returns:
            PIL Image: Image with drawn hairs.
        """
        n_hairs = random.randint(0, self.hairs)
        
        if not n_hairs:
            return img
        
        height, width, _ = img.shape  # target image width and height
        hair_images = [im for im in os.listdir(self.hairs_folder) if 'png' in im]
        
        for _ in range(n_hairs):
            hair = cv2.imread(os.path.join(self.hairs_folder, random.choice(hair_images)))
            hair = cv2.flip(hair, random.choice([-1, 0, 1]))
            hair = cv2.rotate(hair, random.choice([0, 1, 2]))

            h_height, h_width, _ = hair.shape  # hair image width and height
            roi_ho = random.randint(0, img.shape[0] - hair.shape[0])
            roi_wo = random.randint(0, img.shape[1] - hair.shape[1])
            roi = img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

            # Creating a mask and inverse mask
            img2gray = cv2.cvtColor(hair, cv2.COLOR_BGR2GRAY)
            ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            # Now black-out the area of hair in ROI
            img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

            # Take only region of hair from hair image.
            hair_fg = cv2.bitwise_and(hair, hair, mask=mask)

            # Put hair in ROI and modify the target image
            dst = cv2.add(img_bg, hair_fg)

            img[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst
                
        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(hairs={self.hairs}, hairs_folder="{self.hairs_folder}")'

In [ ]:
class Microscope:
    """
    Cutting out the edges around the center circle of the image
    Imitating a picture, taken through the microscope

    Args:
        p (float): probability of applying an augmentation
    """

    def __init__(self, p: float = 0.5):
        self.p = p

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to apply transformation to.

        Returns:
            PIL Image: Image with transformation.
        """
        if random.random() < self.p:
            circle = cv2.circle((np.ones(img.shape) * 255).astype(np.uint8), # image placeholder
                        (img.shape[0]//2, img.shape[1]//2), # center point of circle
                        random.randint(img.shape[0]//2 - 3, img.shape[0]//2 + 15), # radius
                        (0, 0, 0), # color
                        -1)

            mask = circle - 255
            img = np.multiply(img, mask)
        
        return img

    def __repr__(self):
        return f'{self.__class__.__name__}(p={self.p})'

In [ ]:
class Cutout(object):
    def __init__(self, length=16):
        self.length = length

    def __call__(self, img):
        img = np.array(img)

        mask_val = img.mean()

        top = np.random.randint(0 - self.length//2, img.shape[0] - self.length)
        left = np.random.randint(0 - self.length//2, img.shape[1] - self.length)
        bottom = top + self.length
        right = left + self.length

        top = 0 if top < 0 else top
        left = 0 if left < 0 else top

        img[top:bottom, left:right, :] = mask_val

        #img = Image.fromarray(img)

        return img

In [ ]:
transforms_train = A.Compose([
    A.Transpose(p=0.5),
    A.VerticalFlip(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightness(limit=0.2, p=0.75),
    A.RandomContrast(limit=0.2, p=0.75),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),
        A.GridDistortion(num_steps=5, distort_limit=1.),
        A.ElasticTransform(alpha=3),
    ], p=0.7),

    A.CLAHE(clip_limit=4.0, p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    A.Resize(image_size, image_size),
    A.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),    
    A.Normalize()
])

transforms_val = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize()
])

In [ ]:

class MelanomaDataset(Dataset):
    def __init__(self, df: pd.DataFrame, imfolder: str, train: bool = True, transforms = None, meta_features = None):
        """
        Class initialization
        Args:
            df (pd.DataFrame): DataFrame with data description
            imfolder (str): folder with images
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            meta_features (list): list of features with meta information, such as sex and age
            
        """
        self.df = df
        self.imfolder = imfolder
        self.transforms = transforms
        self.train = train
        self.meta_features = meta_features
        
    def __getitem__(self, index):
        if self.train:
            im_path = os.path.join(self.imfolder, self.df.iloc[index]['image_id'] + '.jpg')
        else:
            im_path = os.path.join(self.imfolder, self.df.iloc[index]['image_name'] + '.jpg')
        if not os.path.exists(im_path):
            raise Exception(f"ファイルパスが存在しません：{im_path}")
        x = cv2.imread(im_path)
        meta = np.array(self.df.iloc[index][self.meta_features].values, dtype=np.float32)

        if self.transforms:
            x = self.transforms(image = x)
            x = x['image'].astype(np.float32)
        x = x.transpose(2, 0, 1) #channel first
        if self.train:
            y = self.df.iloc[index]['target']
            return (x, meta), y
        else:
            return (x, meta)
    
    def __len__(self):
        return len(self.df)

In [ ]:
dataset_show = MelanomaDataset(train_df, train_images, train=True,transforms = transforms_train,  meta_features=meta_features)
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        img, meta = img[0],img[1]
        img = np.asarray(img)
        img = img.transpose(1,2,0)
        #img = img.astype(np.uint8)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #rgb→bgr
        axarr[p].imshow(img) 
        axarr[p].set_title(str(label))

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

In [ ]:
def set_bn_eval(module):
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
        module.eval()

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    model.train()
    model.apply(set_bn_eval)
    bar = tqdm(iterator) if Progress_Bar else iterator
    
    for (x, y) in bar:
        #x[0]:image deta, x[1]:table data(meta data)
        x[0] = torch.tensor(x[0], device=device, dtype=torch.float32)
        x[1] = torch.tensor(x[1], device=device, dtype=torch.float32)
        y = torch.tensor(y, device=device, dtype=torch.float32)
        
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y.unsqueeze(1)) #ここで、yにy.unsqeeze()次元拡張を挟む必要がある？(BCEだったら、だと思われる)
        loss.backward()
        optimizer.step()
        loss_np = loss.detach().cpu().numpy()
        epoch_loss += loss_np
        if Progress_Bar:
            bar.set_description('Training loss: %.5f' % (loss_np))
        
    return epoch_loss/len(iterator)

def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    preds = []
    preds = np.array(preds)
    targets = []
    targets = np.array(targets)
    model.eval()
    bar = tqdm(iterator) if Progress_Bar else iterator
    
    with torch.no_grad():
        
        for (x, y) in bar:
        
            x[0] = torch.tensor(x[0], device=device, dtype=torch.float32)
            x[1] = torch.tensor(x[1], device=device, dtype=torch.float32)
            y = torch.tensor(y, device=device, dtype=torch.float32)
            
            y_pred = model(x)
            loss = criterion(y_pred, y.unsqueeze(1))
            loss_np = loss.detach().cpu().numpy()
            epoch_loss += loss_np
            y_pred = torch.sigmoid(y_pred)
            preds = np.append(preds, y_pred.detach().cpu().numpy())
            targets = np.append(targets, y.detach().cpu().numpy())
            
            if Progress_Bar:
                bar.set_description('Validation loss: %.5f' % (loss_np))
    
    val_acc = accuracy_score(targets, np.round(preds))
    try:
       val_roc = roc_auc_score(targets, preds)
    except ValueError:
       val_roc = -1
    
            
    return epoch_loss/len(iterator), val_acc,val_roc

In [ ]:
def fit_model(model, model_name, train_iterator, valid_iterator, optimizer, loss_criterion, device, epochs):
    """ Fits a dataset to model"""
    best_valid_score = float('inf')
    
    train_losses = []
    valid_losses = []
    valid_roc_scores = []
    
    for epoch in range(epochs):
        scheduler.step(epoch)
        start_time = time.time()
    
        train_loss = train(model, train_iterator, optimizer, loss_criterion, device)
        valid_loss, valid_acc_score, valid_roc_score = evaluate(model, valid_iterator, loss_criterion, device)
        
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        valid_roc_scores.append(valid_roc_score)

        if valid_roc_score < best_valid_score:
            best_valid_score = valid_roc_score
            if model_save_path:
                torch.save(model.state_dict(), os.path.join(model_save_path,f'{model_name}.pt'))
            else:
                torch.save(model.state_dict(), f'{model_name}_best.pt')
        
        #schedulerの処理 cosineannealingは別
        #if scheduler != None:
        #    scheduler.step(valid_loss)
        end_time = time.time()

        epoch_mins, epoch_secs = (end_time-start_time)//60,round((end_time-start_time)%60)
    
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'Train Loss: {train_loss:.3f}')
        print(f'Val. Loss: {valid_loss:.3f} | Val. ACC Score: {valid_acc_score:.3f} | Val. Metric Score: {valid_roc_score:.4f}')
        print(f'lr:{optimizer.param_groups[0]["lr"]:.7f}')
        
        torch.save(model.state_dict(), f'{model_name}_final.pt')
        
    return train_losses, valid_losses, valid_roc_scores

In [ ]:
tr_loss=[]
val_loss=[]
val_roc=[]
models = []
for fold in range(1): #now 5 folds
    print(f"Fitting on Fold {fold+1}")
    #Make Train and Valid DataFrame from fold
    train_df_fold = train_df[train_df['fold'] != fold].reset_index(drop=True)
    valid_df_fold = train_df[train_df['fold'] == fold].reset_index(drop=True)
    
    #Build and load Dataset
    train_data = MelanomaDataset(train_df_fold, train_images, train=True, transforms = transforms_train, meta_features=meta_features) 
    valid_data = MelanomaDataset(valid_df_fold, train_images, train=True, transforms = transforms_val, meta_features=meta_features) 
    
    train_iterator = DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=num_workers)
    valid_iterator = DataLoader(valid_data, shuffle=False, batch_size=16, num_workers=num_workers)
    
    model = pre_models[0].to(device)
    loss_criterion = nn.BCEWithLogitsLoss()
    opt= RAdam(model.parameters(),lr=init_lr)
    scheduler = CosineAnnealingLR(opt, cosine_t)
    name = model_name + "_" + enet_type + "_f" + str(fold)
    
    temp_tr_loss, temp_val_loss, temp_val_roc = fit_model(model, name, train_iterator, valid_iterator, opt, loss_criterion, device, epochs=n_epochs)
    
    tr_loss+=temp_tr_loss
    val_loss+=temp_val_loss
    val_roc+=temp_val_roc
    
    models.append(model)

In [ ]:
fig,ax = plt.subplots(nrows=1, ncols=2, figsize=(20,5))
ax[0].plot(tr_loss)
ax[0].set_title('Training and Validation Loss')
ax[0].plot(val_loss)
ax[0].set_xlabel('Epoch')

ax[1].plot(val_roc)
ax[1].set_title('Val ROC Score')
ax[1].set_xlabel('Epoch')


ax[0].legend();
ax[1].legend();

# submit section from here  

In [ ]:
test = MelanomaDataset(df=test_df,
                       imfolder=test_images, 
                       train=False,
                       transforms=transforms_train,  # if TTA. change here
                       meta_features=meta_features)


In [ ]:
#(model数*TTA数)回すので注意
def get_predictions(model, iterator, device):
    
    preds = np.array([0.]*len(test_df))
    model.eval()
    bar = tqdm(iterator) if Progress_Bar else iterator
    
    with torch.no_grad():
        for tta in range(TTA):
            res = np.array([])
            for x in bar:
                x[0] = torch.tensor(x[0], device=device, dtype=torch.float32)
                x[1] = torch.tensor(x[1], device=device, dtype=torch.float32)
                y_pred = model(x)
                y_pred = torch.sigmoid(y_pred)
                res = np.append(res, y_pred.detach().cpu().numpy())
            preds += res
    preds /= TTA
    return preds

In [ ]:
prediction = np.array([0.]*len(test_df))
for i in range(len(models)):
    test_iterator = DataLoader(dataset=test, batch_size=16, shuffle=False, num_workers=num_workers)
    preds = get_predictions(models[i], test_iterator, device)
    prediction += preds
prediction /= len(models)

In [ ]:
sub_df = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
sub_df = sub_df[:50] if DEBUG else sub_df
sub_df['target'] = prediction

sub_df.to_csv('submission.csv', index=False)
sub_df.head()